In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth',None)
import collections
from collections import Counter
import re
from nltk import word_tokenize
import warnings
warnings.filterwarnings("ignore")

# Loading data

In [2]:
hindi_true_df = pd.read_csv('hindi_true.csv')
hindi_true_df = hindi_true_df.rename(columns={'Articles': 'text'})
hindi_true_df['flag'] = 0

hindi_fake_df = pd.read_csv('hindi_fake.csv')
hindi_fake_df = hindi_fake_df.rename(columns={'Articles': 'text'})
hindi_fake_df['flag'] = 1
hindi_fake_df = hindi_fake_df.drop(columns=['Unnamed: 1'])

In [3]:
hindi_news_data = pd.concat([hindi_true_df, hindi_fake_df])

In [4]:
hindi_news_data.head()

,text,flag
0,"सोशल मीडिया कंपनियों को सरकार की चेतावनी, कानून मंत्री बोले- संविधान का सम्मान करना होगा",0
1,"राज्यसभा में रविशंकर प्रसाद ने कहा- सोशल मीडिया से अफवाह फैलाने की इजाजत नहीं, कार्रवाई होगी",0
2,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,0
3,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,0
4,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,0


In [5]:
len(hindi_news_data)

7111

# cleaning text

In [6]:
def removeSpecialCharacters(txt):
    # Compile the regular expression
    regex = re.compile(r'[^\u0900-\u097F\s]')

    # Remove the special characters, numbers, and punctuation marks from the text
    cleaned_text = regex.sub("", txt)
    
    return cleaned_text

def tokenizeSentence(txt):
    return word_tokenize(txt)

def removeStopWords(sentence):
    # Define the list of hindi stop words
    hindi_stop_words = ['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने', 'हमारा', 'अपना', 'हम', 'आप', 'आपका', 'तुम्हारा', 
                        'अपने आप', 'स्वयं', 'वह', 'इसे', 'उसके', 'खुद को', 'कि वह', 'उसकी', 'उसका', 'खुद ही', 'यह', 
                        'इसके', 'उन्होने', 'अपने', 'क्या', 'जो', 'किसे', 'किसको', 'कि', 'ये', 'हूँ', 'होता है', 'रहे', 'थी', 
                        'थे', 'होना', 'गया', 'किया जा रहा है', 'किया है', 'है', 'पडा', 'होने', 'करना', 'करता है', 'किया', 
                        'रही', 'एक', 'लेकिन', 'अगर', 'या', 'क्यूंकि', 'जैसा', 'जब तक', 'जबकि', 'की', 'पर', 'द्वारा', 
                        'के लिए', 'साथ', 'के बारे में', 'खिलाफ', 'बीच', 'में', 'के माध्यम से', 'दौरान', 'से पहले', 'के बाद', 
                        'ऊपर', 'नीचे', 'को', 'से', 'तक', 'से नीचे', 'करने में', 'निकल', 'बंद', 'से अधिक', 'तहत', 'दुबारा', 
                        'आगे', 'फिर', 'एक बार', 'यहाँ', 'वहाँ', 'कब', 'कहाँ', 'क्यों', 'कैसे', 'सारे', 'किसी', 'दोनो', 'प्रत्येक', 
                        'ज्यादा', 'अधिकांश', 'अन्य', 'में कुछ', 'ऐसा', 'में कोई', 'मात्र', 'खुद', 'समान', 'इसलिए', 'बहुत', 'सकता', 
                        'जायेंगे', 'जरा', 'चाहिए', 'अभी', 'और', 'कर दिया', 'रखें', 'का', 'हैं', 'इस', 'होता', 'करने', 'ने', 'बनी', 
                        'तो', 'ही', 'हो', 'इसका', 'था', 'हुआ', 'वाले', 'बाद', 'लिए', 'सकते', 'इसमें', 'दो', 'वे', 'करते', 
                        'कहा', 'वर्ग', 'कई', 'करें', 'होती', 'अपनी', 'उनके', 'यदि', 'हुई', 'जा', 'कहते', 'जब', 'होते', 'कोई', 
                        'हुए', 'व', 'जैसे', 'सभी', 'करता', 'उनकी', 'तरह', 'उस', 'आदि', 'इसकी', 'उनका', 'इसी', 'पे', 'तथा', 
                        'भी', 'परंतु', 'इन', 'कम', 'दूर', 'पूरे', 'गये', 'तुम', 'मै', 'यहां', 'हुये', 'कभी', 'अथवा', 'गयी', 'प्रति', 
                        'जाता', 'इन्हें', 'गई', 'अब', 'जिसमें', 'लिया', 'बड़ा', 'जाती', 'तब', 'उसे', 'जाते', 'लेकर', 'बड़े', 'दूसरे', 
                        'जाने', 'बाहर', 'स्थान', 'उन्हें ', 'गए', 'ऐसे', 'जिससे', 'समय', 'दोनों', 'किए', 'रहती', 'इनके', 'इनका', 
                        'इनकी', 'सकती', 'आज', 'कल', 'जिन्हें', 'जिन्हों', 'तिन्हें', 'तिन्हों', 'किन्हों', 'किन्हें', 'इत्यादि', 'इन्हों', 
                        'उन्हों', 'बिलकुल', 'निहायत', 'इन्हीं', 'उन्हीं', 'जितना', 'दूसरा', 'कितना', 'साबुत', 'वग़ैरह', 'कौनसा', 
                        'लिये', 'दिया', 'जिसे', 'तिसे', 'काफ़ी', 'पहले', 'बाला', 'मानो', 'अंदर', 'भीतर', 'पूरा', 'सारा', 
                        'उनको', 'वहीं', 'जहाँ', 'जीधर', '\ufeffके', 'एवं', 'कुछ', 'कुल', 'रहा', 'जिस', 'जिन', 'तिस', 
                        'तिन', 'कौन', 'किस', 'संग', 'यही', 'बही', 'उसी', 'मगर', 'कर', 'मे', 'एस', 'उन', 'सो', 'अत']

    tokens = word_tokenize(sentence)
    # Remove the hindi stop words from the tokens
    return " ".join([token for token in tokens if token not in hindi_stop_words])

# Stemming
suffixes = {
    1: ["ो", "े", "ू", "ु", "ी", "ि", "ा"],
    2: ["कर", "ाओ", "िए", "ाई", "ाए", "ने", "नी", "ना", "ते", "ीं", "ती", "ता", "ाँ", "ां", "ों", "ें"],
    3: ["ाकर", "ाइए", "ाईं", "ाया", "ेगी", "ेगा", "ोगी", "ोगे", "ाने", "ाना", "ाते", "ाती", "ाता", "तीं", "ाओं", "ाएं", "ुओं", "ुएं", "ुआं"],
    4: ["ाएगी", "ाएगा", "ाओगी", "ाओगे", "एंगी", "ेंगी", "एंगे", "ेंगे", "ूंगी", "ूंगा", "ातीं", "नाओं", "नाएं", "ताओं", "ताएं", "ियाँ", "ियों", "ियां"],
    5: ["ाएंगी", "ाएंगे", "ाऊंगी", "ाऊंगा", "ाइयाँ", "ाइयों", "ाइयां"],
}

def stemWord(word):
    for L in 5, 4, 3, 2, 1:
        if len(word) > L + 1:
            for suf in suffixes[L]:
                if word.endswith(suf):
                    return word[:-L]
    return word


def performStemming(sentence):
    words = word_tokenize(sentence)
    for ind in range(len(words)):
        words[ind] = stemWord(words[ind])
        
    return " ".join(words)
    

In [7]:
def preprocessText():
    #remove stop words
    hindi_news_data['text'] = hindi_news_data["text"].apply(removeSpecialCharacters)
    
    #remove stop words
    hindi_news_data['clean_txt'] = hindi_news_data["text"].apply(removeStopWords)
    
    #stemming
    hindi_news_data['clean_txt'] = hindi_news_data["clean_txt"].apply(performStemming)
    
    #tokenize
    hindi_news_data['tokens'] = hindi_news_data["clean_txt"].apply(tokenizeSentence)
    

preprocessText()
hindi_news_data.head()   

,text,flag,clean_txt,tokens
0,सोशल मीडिया कंपनियों को सरकार की चेतावनी कानून मंत्री बोले संविधान का सम्मान करना होगा,0,सोशल मीडिय कंपन सरकार चेताव कानून मंत्र बोल संविधान सम्मान होग,"[सोशल, मीडिय, कंपन, सरकार, चेताव, कानून, मंत्र, बोल, संविधान, सम्मान, होग]"
1,राज्यसभा में रविशंकर प्रसाद ने कहा सोशल मीडिया से अफवाह फैलाने की इजाजत नहीं कार्रवाई होगी,0,राज्यसभ रविशं प्रसाद सोशल मीडिय अफवाह फैल इजाजत नह कार्रव होग,"[राज्यसभ, रविशं, प्रसाद, सोशल, मीडिय, अफवाह, फैल, इजाजत, नह, कार्रव, होग]"
2,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,0,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,"[बिहार, के, जातिगत, समीकरण, बात, लंब, बीजेप, हिंद, पहचान, के, आधार, वोटर, लामबंद, लग, इसक, सोशल, इंजीनियरिंग, उसक, ध्यान, रह]"
3,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,0,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,"[भारत, ऑस्ट्रेलिय, टेस्ट, सिरीज़, ऐतिहासिक, जीत, दर्ज, चौथ, निर्णायक, मैच, नौजवान, गेंदबाज़, मोहम्मद, सिराज, अहम, भूमिक, निभ]"
4,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,0,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,"[ब्रिस्बेन, टेस्ट, भारत, दीवार, कह, वाल, चेतेश्वर, पुजार, अर्धशतक, जम, पार, के, वो, चोटिल, ऑस्ट्रेलिय, गेंदबाज़, गेंद, उनक, शरीर, लग]"


In [8]:
hindi_news_data.drop(columns=['flag']).head()

,text,clean_txt,tokens
0,सोशल मीडिया कंपनियों को सरकार की चेतावनी कानून मंत्री बोले संविधान का सम्मान करना होगा,सोशल मीडिय कंपन सरकार चेताव कानून मंत्र बोल संविधान सम्मान होग,"[सोशल, मीडिय, कंपन, सरकार, चेताव, कानून, मंत्र, बोल, संविधान, सम्मान, होग]"
1,राज्यसभा में रविशंकर प्रसाद ने कहा सोशल मीडिया से अफवाह फैलाने की इजाजत नहीं कार्रवाई होगी,राज्यसभ रविशं प्रसाद सोशल मीडिय अफवाह फैल इजाजत नह कार्रव होग,"[राज्यसभ, रविशं, प्रसाद, सोशल, मीडिय, अफवाह, फैल, इजाजत, नह, कार्रव, होग]"
2,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,"[बिहार, के, जातिगत, समीकरण, बात, लंब, बीजेप, हिंद, पहचान, के, आधार, वोटर, लामबंद, लग, इसक, सोशल, इंजीनियरिंग, उसक, ध्यान, रह]"
3,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,"[भारत, ऑस्ट्रेलिय, टेस्ट, सिरीज़, ऐतिहासिक, जीत, दर्ज, चौथ, निर्णायक, मैच, नौजवान, गेंदबाज़, मोहम्मद, सिराज, अहम, भूमिक, निभ]"
4,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,"[ब्रिस्बेन, टेस्ट, भारत, दीवार, कह, वाल, चेतेश्वर, पुजार, अर्धशतक, जम, पार, के, वो, चोटिल, ऑस्ट्रेलिय, गेंदबाज़, गेंद, उनक, शरीर, लग]"


# Feature Engineering

## adding length of the string

In [9]:
def count_length():
    hindi_news_data['word_count'] = hindi_news_data['text'].apply(lambda x: len(str(x).split(" ")))
count_length()


hindi_news_data.head()

,text,flag,clean_txt,tokens,word_count
0,सोशल मीडिया कंपनियों को सरकार की चेतावनी कानून मंत्री बोले संविधान का सम्मान करना होगा,0,सोशल मीडिय कंपन सरकार चेताव कानून मंत्र बोल संविधान सम्मान होग,"[सोशल, मीडिय, कंपन, सरकार, चेताव, कानून, मंत्र, बोल, संविधान, सम्मान, होग]",15
1,राज्यसभा में रविशंकर प्रसाद ने कहा सोशल मीडिया से अफवाह फैलाने की इजाजत नहीं कार्रवाई होगी,0,राज्यसभ रविशं प्रसाद सोशल मीडिय अफवाह फैल इजाजत नह कार्रव होग,"[राज्यसभ, रविशं, प्रसाद, सोशल, मीडिय, अफवाह, फैल, इजाजत, नह, कार्रव, होग]",16
2,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,0,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,"[बिहार, के, जातिगत, समीकरण, बात, लंब, बीजेप, हिंद, पहचान, के, आधार, वोटर, लामबंद, लग, इसक, सोशल, इंजीनियरिंग, उसक, ध्यान, रह]",21
3,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,0,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,"[भारत, ऑस्ट्रेलिय, टेस्ट, सिरीज़, ऐतिहासिक, जीत, दर्ज, चौथ, निर्णायक, मैच, नौजवान, गेंदबाज़, मोहम्मद, सिराज, अहम, भूमिक, निभ]",18
4,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,0,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,"[ब्रिस्बेन, टेस्ट, भारत, दीवार, कह, वाल, चेतेश्वर, पुजार, अर्धशतक, जम, पार, के, वो, चोटिल, ऑस्ट्रेलिय, गेंदबाज़, गेंद, उनक, शरीर, लग]",21


## unique tokens in each article

In [10]:
hindi_news_data['unique_words'] = hindi_news_data['tokens'].apply(set).apply(len)
hindi_news_data.head()

,text,flag,clean_txt,tokens,word_count,unique_words
0,सोशल मीडिया कंपनियों को सरकार की चेतावनी कानून मंत्री बोले संविधान का सम्मान करना होगा,0,सोशल मीडिय कंपन सरकार चेताव कानून मंत्र बोल संविधान सम्मान होग,"[सोशल, मीडिय, कंपन, सरकार, चेताव, कानून, मंत्र, बोल, संविधान, सम्मान, होग]",15,11
1,राज्यसभा में रविशंकर प्रसाद ने कहा सोशल मीडिया से अफवाह फैलाने की इजाजत नहीं कार्रवाई होगी,0,राज्यसभ रविशं प्रसाद सोशल मीडिय अफवाह फैल इजाजत नह कार्रव होग,"[राज्यसभ, रविशं, प्रसाद, सोशल, मीडिय, अफवाह, फैल, इजाजत, नह, कार्रव, होग]",16,11
2,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,0,बिहार के जातिगत समीकरण बात लंब बीजेप हिंद पहचान के आधार वोटर लामबंद लग इसक सोशल इंजीनियरिंग उसक ध्यान रह,"[बिहार, के, जातिगत, समीकरण, बात, लंब, बीजेप, हिंद, पहचान, के, आधार, वोटर, लामबंद, लग, इसक, सोशल, इंजीनियरिंग, उसक, ध्यान, रह]",21,19
3,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,0,भारत ऑस्ट्रेलिय टेस्ट सिरीज़ ऐतिहासिक जीत दर्ज चौथ निर्णायक मैच नौजवान गेंदबाज़ मोहम्मद सिराज अहम भूमिक निभ,"[भारत, ऑस्ट्रेलिय, टेस्ट, सिरीज़, ऐतिहासिक, जीत, दर्ज, चौथ, निर्णायक, मैच, नौजवान, गेंदबाज़, मोहम्मद, सिराज, अहम, भूमिक, निभ]",18,17
4,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,0,ब्रिस्बेन टेस्ट भारत दीवार कह वाल चेतेश्वर पुजार अर्धशतक जम पार के वो चोटिल ऑस्ट्रेलिय गेंदबाज़ गेंद उनक शरीर लग,"[ब्रिस्बेन, टेस्ट, भारत, दीवार, कह, वाल, चेतेश्वर, पुजार, अर्धशतक, जम, पार, के, वो, चोटिल, ऑस्ट्रेलिय, गेंदबाज़, गेंद, उनक, शरीर, लग]",21,20


# train test split

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(hindi_news_data.drop(columns=['flag', 'text', 'tokens']), hindi_news_data['flag'], test_size=0.2, random_state=40, stratify=hindi_news_data['flag'])

In [12]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5688, 3), (5688,), (1423, 3), (1423,))

In [13]:
X_train.head()

,clean_txt,word_count,unique_words
2546,आरक्षण सूच जार ग्राम प्रधान जातिवार लिस्ट वायरल मच हड़कंप,21,10
212,आख़िर ऐस क्य कृष पंजाब उत्तर प्रदेश अधिक भागीदार वाल राज्य बिहार किसान नह क्य कृष सब बढ़िय चल रह,21,18
2716,श्रीलंक आर्थिक संकट राष्ट्रपत गोटब के अप्रैल आ अविश्वास प्रस्ताव राजपक्ष हट अपील,24,13
1277,मार्च ले मई मह अचानक एक्टिव फेक न्यूज वेबसाइटस हैंउनम पेइंग गेस्ट के,14,13
2293,कोरो के चल जोड़ हनीमून बन क़ैद,10,7


In [14]:
y_train.head()

2546    1
212     0
2716    0
1277    1
2293    0
Name: flag, dtype: int64

# Text vectorization 

## BOW

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
bow = dict()
bow["train"] = [vectorizer.fit_transform(X_train['clean_txt']), X_train['word_count'], y_train]
bow["test"] = [vectorizer.transform(X_test['clean_txt']), X_test['word_count'], y_test]


In [16]:
print(bow["train"][0].shape)
print(bow["test"][0].shape)

(5688, 1914)
(1423, 1914)


## TFIDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
transformer = TfidfVectorizer()
tfidf = dict()
tfidf["train"] = [transformer.fit_transform(X_train['clean_txt']), X_train['word_count'], y_train]
tfidf["test"] = [transformer.transform(X_test['clean_txt']), X_test['word_count'], y_test]

In [18]:
print(tfidf["train"][0].shape)
print(tfidf["test"][0].shape)

(5688, 1914)
(1423, 1914)


# Classification

## Hyperparameter Tuning

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error, log_loss
from tabulate import tabulate

def perform_grid_search(classifiers, param_values, X, y):
    table = [['Classifier Name', 'Best Params', 'Accuracy', 'F1', 'Log loss']]
    for classifier, params in zip(classifiers, param_values):
        # create a grid search object with the specified parameters
        grid_search = GridSearchCV(classifier, params, scoring="accuracy", cv=3)
        
        X_train, X_cv, y_train, y_cv = train_test_split(X, y, 
                                                        test_size=0.2, random_state=40, 
                                                        stratify=y)
        
        # perform grid search on the training data
        grid_search.fit(X_train, y_train)

        # retrieve the best estimator from the grid search and compute f1 and RMSE
        best_estimator = grid_search.best_estimator_
        y_pred = best_estimator.predict(X_cv)
        f1 = f1_score(y_cv, y_pred)
        ll = log_loss(y_cv, y_pred)
        
        table.append([classifier.__class__.__name__, grid_search.best_params_, grid_search.best_score_, f1, ll])
    
    #print results
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [20]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

classifiers = [
    GaussianNB(),
    KNeighborsClassifier(),
    LogisticRegression(),
    SVC(), 
    RandomForestClassifier(),
    SGDClassifier(),
    GradientBoostingClassifier(),
    xgb.XGBClassifier(eval_metric='mlogloss')
]
params = [
    {
    'priors': [None, [0.25, 0.75], [0.5, 0.5], [0.75, 0.25]],
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]},
    {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'chebyshev']
    },
    {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 300, 400, 500]
    },
    {
    'C': [0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001]
    }, 
    {
    'n_estimators': [10, 20, 30, 40, 50],
    'criterion': ['gini', 'entropy'],
    'max_depth': [5, 10, 15, 20, 25],
    'min_samples_split': [2, 4, 6, 8, 10]
    },
    {
    'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
    'max_iter': [100, 200, 300, 400, 500]
    },
    {
    'n_estimators': [10, 20, 30, 40, 50],
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'max_depth': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 4, 6, 8, 10]
    },
    {
        'max_depth': [5, 10, 15],
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.9, 1, 1.2, 1.3, 1.4],
        'gamma': [0.1, 0.01, 1]
    }]

### Analysis on BOW

In [21]:
perform_grid_search(classifiers, params, bow["train"][0].todense(), bow["train"][2])

╒══════════════════════════════╤═══════════════════════════════════════════════════╤════════════╤══════╤════════════╕
│ Classifier Name              │ Best Params                                       │   Accuracy │   F1 │   Log loss │
╞══════════════════════════════╪═══════════════════════════════════════════════════╪════════════╪══════╪════════════╡
│ GaussianNB                   │ prior=[0.75, 0.25], var_smooting=0.01             │       0.72 │ 0.8  │       8.59 │
├──────────────────────────────┼───────────────────────────────────────────────────┼────────────┼──────┼────────────┤
│ K-nearest Neighbors          │ metric=euclidean, n_neighbors=5, weights=distance │       0.79 │ 0.81 │       6.8  │
├──────────────────────────────┼───────────────────────────────────────────────────┼────────────┼──────┼────────────┤
│ Logistic Regression          │ C=1, max_iter=100, penalty=l2, solver=liblinear   │       0.84 │ 0.83 │       6.07 │
├──────────────────────────────┼────────────────────────

### Analysis on TFIDF

In [22]:
perform_grid_search(classifiers, params, tfidf["train"][0].todense(), tfidf["train"][2])

╒══════════════════════════════╤═══════════════════════════════════════════════════╤════════════╤══════╤════════════╕
│ Classifier Name              │ Best Params                                       │   Accuracy │   F1 │   Log loss │
╞══════════════════════════════╪═══════════════════════════════════════════════════╪════════════╪══════╪════════════╡
│ GaussianNB                   │ prior=[0.75, 0.25], var_smooting=0.1              │       0.75 │ 0.82 │       7.56 │
├──────────────────────────────┼───────────────────────────────────────────────────┼────────────┼──────┼────────────┤
│ K-nearest Neighbors          │ metric=chebyshev, n_neighbors=7, weights=distance │       0.66 │ 0.65 │      11.96 │
├──────────────────────────────┼───────────────────────────────────────────────────┼────────────┼──────┼────────────┤
│ Logistic Regression          │ C=1, max_iter=100, penalty=l2, solver=saga        │       0.84 │ 0.84 │       5.71 │
├──────────────────────────────┼────────────────────────

# setting up ensemble model

In [26]:
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

def getEnsembleResults(text_input, labels, isBOW):
    # Define the models to use in the ensemble
    if isBOW: 
        model1 = xgb.XGBClassifier(eval_metric='mlogloss', gamma=0.01, max_depth=5, n_estimator=100)
        model2 = SVC(C=1, gamma=0.1)
        model3 = RandomForestClassifier(criterion='gini', min_samples_split=6, 
                               max_depth=20, n_estimators=30)
    else:
        model1 = xgb.XGBClassifier(eval_metric='mlogloss', gamma=0.1, max_depth=5, n_estimator=50)
        model2 = SVC(C=10, gamma=1)
        model3 = RandomForestClassifier(criterion='gini', min_samples_split=4, 
                               max_depth=25, n_estimators=50)

    # Define the ensemble model
    ensemble_model = VotingClassifier(estimators=[('LR', model1), ('SVC', model2), ('RF', model3)])

    X_train, X_cv, y_train, y_cv = train_test_split(text_input, labels, 
                                                        test_size=0.2, random_state=40, 
                                                        stratify=labels)
    
    
    # Fit the model
    ensemble_model.fit(X_train, y_train)

    # Import necessary libraries
    from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, log_loss

    # Predict labels for the ensemble model
    predictions = ensemble_model.predict(X_cv)

    # Calculate and print the accuracy
    accuracy = accuracy_score(y_cv, predictions)
    print("Accuracy:", accuracy)

    # Calculate and print the F1 score
    f1 = f1_score(y_cv, predictions)
    print("F1 score:", f1)

    # Calculate and print the log loss
    log_loss_value = log_loss(y_cv, predictions)
    print("Log loss:", log_loss_value)


# Evaluating ensemble model

In [24]:
getEnsembleResults(bow["train"][0].todense(), bow["train"][2], True)

Accuracy: 0.8588576449912126
F1 score: 0.8749747048903878
Log loss: 3.920314161485456


In [25]:
getEnsembleResults(tfidf["train"][0].todense(), tfidf["train"][2], False)

Accuracy: 0.8776449912126538
F1 score: 0.8922448979591838
Log loss: 3.616802948898982
